In [1]:
# pip install pydicom
# pip install numpy
# pip install matplotlib
# pip install opencv-python
# pip install tqdm
# pip install pandas
# pip install glob2

In [2]:
import DicomPipeLine 
import cv2

In [3]:
main_data_source_path = 'F:\DICOM data'

In [4]:
preprocessed_data_path = 'F:\Final Year Project\Data Pipeline\TestDataBase_2'

In [5]:
sub_folders_to_scan_for = ['2','3','4','5']

In [6]:
path_obj = DicomPipeLine.Path_Settings(main_data_source_path, preprocessed_data_path, sub_folders_to_scan_for)

100%|███████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 17211.90it/s]

- Found 49 folders with a total of 191 scans


In [7]:
# these are the attributes to extract from each patient
dcm_attributes_level_0 = [
    'PatientName',
    'PatientBirthDate',
    'PatientSex',
    'PatientSize',
    'PatientWeight',
    'StudyDate', 
    'StudyTime',
    'InstitutionAddress', 
    'InstitutionName',
    'Manufacturer', 
    'ManufacturerModelName'
]

# these are the attributes to extract from each scan per patient
dcm_attributes_level_1 = [
    'AcquisitionDate', 
    'AcquisitionTime', 
    'PatientPosition',
    'SeriesDescription',
    'SeriesDate',
    'SeriesTime'
]

# these are the attributes to extract from each frame per scan 
dcm_attributes_level_2 = [
    'Rows',
    'Columns',
    'InstanceNumber',
    'PixelSpacing',
    'ImagePositionPatient',
    'ImageOrientationPatient',
    'SliceLocation',
    'SliceThickness'
]

# specify all 3 levels of attributes in a array
dcm_attributes = [dcm_attributes_level_0, dcm_attributes_level_1, dcm_attributes_level_2]


In [8]:
dcm_to_numpy_obj = DicomPipeLine.DCM_Input_To_NPY_Output(path_obj, dcm_attributes)

In [ ]:
dcm_to_numpy_obj.iterate(2)

- processing folder F:\DICOM data\1.3.12.2.1107.5.1.4.11050.30000020121704410612500000013


100%|███████████████████████████████████████████████████████████████████████████████| 333/333 [00:02<00:00, 142.11it/s]


- Saving Processed scans ['PET WB  (AC)', 'CT PLAIN', 'CT ARTERIAL', 'CT VENOUS'] ....


In [8]:
streamer = DicomPipeLine.Stream_Data(path_obj)

In [14]:
import time

def normalize(img_array):
    return cv2.normalize(img_array, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)

In [16]:
for ct_frame, pt_frame in zip(streamer.iter('1', 'CT PLAIN'), streamer.iter('1', 'PET WB  (AC)')):
    
    ct_frame = normalize(ct_frame)
    pt_frame = normalize(pt_frame)
    
    cv2.imshow('CT Image', ct_frame)
    cv2.imshow('PET Image', pt_frame)
    
    time.sleep(.2)
    cv2.waitKey(4)

F:\Final Year Project\Data Pipeline\TestDataBase_2\1\CT PLAIN.pickle.gzip
F:\Final Year Project\Data Pipeline\TestDataBase_2\1\PET WB  (AC).pickle.gzip


In [ ]:
# prev_folder = None
# prev_scan = None
# for img_array, folder_name, scan_name in streamer.iterate_image_data(0, 1):
    
#     img_array = cv2.normalize(img_array, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)
#     cv2.imshow('View Window',img_array)
    
#     if folder_name != prev_folder or prev_scan != scan_name:
#         print(f'folder : {folder_name} | Scan : {scan_name}')
#         prev_folder = folder_name
#         prev_scan = scan_name

#     cv2.waitKey(2)